In [5]:
import numpy as np
import pickle
import os
import shutil
import glob

In [5]:
# Storing the 10k model
long_model = np.load('../agent_code/agent_vq10/logs/Q_data/Q9483.npy')
with open('../agent_code/agent_vq10/other_models/model_vq10_10k.pt', "wb") as file:
        pickle.dump(long_model, file)

In [14]:
def collect_Q_data (agent_name, model_name, number_of_rounds):
    source_folder = f"../agent_code/agent_{agent_name}/logs/Q_data/"
    Q_file_name = lambda x: f"{source_folder}Q{x}.npy"
    Q_file      = lambda x: np.load(Q_file_name(x))
  
    all_rounds = np.arange(number_of_rounds) + 1
    allQ       = np.zeros((number_of_rounds, *Q_file(1).shape))
    for round in all_rounds:
        print(f"Loading 'Q{round}'", end="\r")
        allQ[round-1] = Q_file(round)
    print('\n')

    analysis_folder = f"../analysis/{agent_name}/{model_name}/"
    allQ_file_name  = f"{analysis_folder}Qtrain.npy"
    # Check to prevent accidental overwrites
    if os.path.exists(allQ_file_name):
        print(f"Do you want to overwrite the Qtrain.py file of agent_{agent_name}_{model_name}? (y/n)")
        answer = ""
        while not (answer == "y" or answer == "n"):  
            answer = input()
        if answer == "n":
            print("Didn't overwrite.")
            return
        else:
            print(f"Overwriting '{allQ_file_name}'.")
    np.save(allQ_file_name, allQ)

    if allQ_file_name in glob.glob(f"{analysis_folder}*.npy"):
        print(f"Removing Q-files in '{source_folder}'.")
        for file in glob.glob(f"{source_folder}Q*.npy"):
            os.remove(file)
    print("Done.")

In [21]:
collect_Q_data("vq10", "10k", 9483)

Do you want to overwrite the Qtrain.py file of agent_vq10_10k? (y/n)
Didn't overwrite.


In [9]:
collect_Q_data("swq13", "train-data", 100)

Done.ng 'Q100'


In [12]:
def copy_model (agent_name, model_name):
    source_file        = f"../agent_code/agent_{agent_name}/model_{agent_name}_{model_name}.pt"
    destination_folder = f"./{agent_name}/{model_name}/"

    shutil.copy(source_file, destination_folder)

In [13]:
copy_model("swq13", "10k")

In [17]:
def move_log (agent_name, model_name, mode = "train"):
    source_file      = f"../agent_code/agent_{agent_name}/logs/agent_{agent_name}.log"
    destination_file = f"./{agent_name}/{model_name}/log_{mode}.log"

    shutil.move(source_file, destination_file)

In [18]:
move_log("swq13", "10k")

In [1]:
def copy_analysis_template (agent_name, model_name, mode = "train"):
    source_file        = f"./templates/analysis_{mode}.ipynb"
    destination_folder = f"./{agent_name}/{model_name}/"

    shutil.copy(source_file, destination_folder)

In [4]:
copy_analysis_template("swq13", "10k")

In [8]:
def move_results (agent_name, model_name, mode = "train"):
    source_file        = glob.glob("../results/*.json")[-1]   # latest json log file
    destination_folder = f"./{agent_name}/{model_name}/"
    destination_file   = f"{destination_folder}results_{mode}.json"

    if destination_file not in glob.glob(destination_folder):
        shutil.move(source_file, destination_file)
    else:
        print(f"There's already a 'results_{mode}.json' file in '{destination_folder}'.")

In [6]:
glob.glob("../results/*.json")

['../results/2022-03-15 20-30-00.json',
 '../results/2022-03-15 21-36-36.json',
 '../results/2022-03-16 00-27-15.json']

In [9]:
move_results("swq13", "10k")

In [15]:
collect_Q_data("swq13", "10k", 10_000)

Loading 'Q10000'

Do you want to overwrite the Qtrain.py file of agent_swq13_10k? (y/n)
Overwriting '../analysis/swq13/10k/Qtrain.npy'.
Removing Q-files in '../agent_code/agent_swq13/logs/Q_data/'.
Done.


In [16]:
def collect_analysis_data (agent_name, model_name, n_rounds, mode = "train"):
    copy_model            (agent_name, model_name)
    move_log              (agent_name, model_name, mode)
    copy_analysis_template(agent_name, model_name, mode)
    move_results          (agent_name, model_name, mode)
    collect_Q_data        (agent_name, model_name, n_rounds)
    